In [ ]:
%reset -f

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:

selection = 'ZO' #ZS OR ZO
if selection == 'ZS':
  components = 12
else:
  components = 8


zof = '/content/drive/My Drive/Colab Notebooks/Analogs/Zonal_{}/Zonal4_Test'.format(selection)
zsf = '/content/drive/My Drive/Colab Notebooks/Analogs/Zonal_{}/ZonalM_Test'.format(selection)


